## [--- Day 11: Reactor ---](https://adventofcode.com/2025/day/11)

In [17]:
import re
from dataclasses import dataclass, field
from collections import deque


@dataclass
class Node:
    name: str
    parents: set["Node"] = field(default_factory=set)
    children: set["Node"] = field(default_factory=set)

    def __hash__(self) -> int:
        return hash(self.name)


def count_paths(start: Node, end: Node, nodes: dict[str, Node]) -> int:
    # Assign graph depth to relevant segment, enabling iteration in topological order
    visited: set[Node] = set([start])
    in_degrees: dict[Node, int] = {}
    stack = [start]
    while stack:
        node = stack.pop()
        if node == end:
            continue
        for child in node.children:
            in_degrees[child] = in_degrees.get(child, 0) + 1
            if child not in visited:
                visited.add(child)
                stack.append(child)

    # Kahn's algorithm. Starting from all degree 0 nodes, but only "you" has a path to propagate
    path_counts: dict[Node, int] = {node: 0 for node in visited}
    path_counts[start] = 1

    queue: deque[Node] = deque([start])
    while queue:
        node = queue.popleft()
        if node == end:
            break
        count = path_counts[node]
        for child in node.children:
            path_counts[child] += count
            in_degrees[child] -= 1
            if in_degrees[child] == 0:
                queue.append(child)

    return path_counts[end]


def solve() -> None:
    with open("..\\data\\11.txt") as file:
        pattern: re.Pattern = re.compile(r"\w{3}")
        nodes: dict[str, Node] = {}
        child_map: dict[str, list[str]] = {}

        for line in file.readlines():
            matches = pattern.findall(line)
            node = Node(name=matches[0])
            child_map[node.name] = matches[1:]
            nodes[node.name] = node

        nodes["out"] = Node(name="out")

    for name, children in child_map.items():
        for child in children:
            nodes[name].children.add(nodes[child])
            nodes[child].parents.add(nodes[name])

    start: Node = nodes["you"]
    end: Node = nodes["out"]

    print(f"Path Count (part 1): {count_paths(start, end, nodes)}")

    if count_paths(nodes["fft"], nodes["dac"], nodes) == 0:
        # A DAG can only have one of the two orders
        waypoint_1 = nodes["dac"]
        waypoint_2 = nodes["fft"]
    else:
        waypoint_1 = nodes["fft"]
        waypoint_2 = nodes["dac"]

    start = nodes["svr"]
    end = nodes["out"]
    num_paths = (
        count_paths(start, waypoint_1, nodes)
        * count_paths(waypoint_1, waypoint_2, nodes)
        * count_paths(waypoint_2, end, nodes)
    )

    print(f"Path Count (part 2): {num_paths}")


solve()

Path Count (part 1): 791
Path Count (part 2): 520476725037672
